In [1]:
# Import libraries
import pandas as pd
import numpy as np
import ast
from toolbox.utils import calculate_U_value
from datetime import datetime

In [2]:
# Import inputs file
scenarios = ['present-day', '2050', '2080']

inputs_dict = {}
for scenario in scenarios:
    path = '../simulations/' + scenario + '/inputs.csv'
    inputs_dict[scenario] = pd.read_csv(path)

# Clean Inputs data
for key in inputs_dict.keys():
    inputs_dict[key].rename(columns={'Unnamed: 0': 'key'}, inplace=True)
    inputs_dict[key]['volume'] = inputs_dict[key]['floor_area'] * inputs_dict[key]['building_height']

    inputs_dict[key]['floor_construction_list'] = inputs_dict[key]['floor_construction_list'].apply(ast.literal_eval)
    inputs_dict[key]['wall_construction_list'] = inputs_dict[key]['wall_construction_list'].apply(ast.literal_eval)
    inputs_dict[key]['roof_construction_list'] = inputs_dict[key]['roof_construction_list'].apply(ast.literal_eval)
    inputs_dict[key]['ground_construction_list'] = inputs_dict[key]['ground_construction_list'].apply(ast.literal_eval)
    
    inputs_dict[key]['floor_U_value'] = inputs_dict[key]['floor_construction_list'].apply(calculate_U_value)
    inputs_dict[key]['wall_U_value'] = inputs_dict[key]['wall_construction_list'].apply(calculate_U_value)
    inputs_dict[key]['roof_U_value'] = inputs_dict[key]['roof_construction_list'].apply(calculate_U_value)
    inputs_dict[key]['ground_U_value'] = inputs_dict[key]['ground_construction_list'].apply(calculate_U_value)

    inputs_dict[key]['door_area'] = inputs_dict[key]['door_width']* inputs_dict[key]['door_height_fraction_building_height'] * inputs_dict[key]['building_height']
    
    inputs_dict[key]['wall_construction'] = inputs_dict[key]['wall_construction_list'].apply(lambda x: 1 if 'Concrete' in str(x) else 0)
    inputs_dict[key]['roof_construction'] = inputs_dict[key]['roof_construction_list'].apply(lambda x: 1 if 'Concrete' in str(x) else 0)
    inputs_dict[key]['ground_construction'] = inputs_dict[key]['ground_construction_list'].apply(lambda x: 1 if 'Concrete' in str(x) else 0)
    inputs_dict[key]['floor_construction'] = inputs_dict[key]['floor_construction_list'].apply(lambda x: 1 if 'Concrete' in str(x) else 0)

    inputs_dict[key]['construction_type'] = inputs_dict[key][['wall_construction', 'roof_construction', 'ground_construction']].mode(axis=1)[0]
    
    inputs_dict[key]['ceiling_insulation'] = inputs_dict[key]['roof_construction_list'].apply(lambda x: 1 if 'Insulation' in str(x) else 0)

    inputs_dict[key] = inputs_dict[key][['key', 'floor_area', 'volume', 'orientation', 'wall_U_value', 'roof_U_value', 'ground_U_value', 'construction_type', 'ceiling_insulation', 'u_factor', 'shgc', 'window_to_wall_ratio', 'door_area', 'number_people', 'activity_level', 'P_equip', 'elec_gas','P_hvac', 'lighting_bulbs', 'infiltration_rate', 'type_house']]
    
    inputs_dict[key].set_index('key', inplace=True)

    print('\n\n Scenario: ' + key)
    print(inputs_dict[key].info())



 Scenario: present-day
<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   floor_area            500 non-null    float64
 1   volume                500 non-null    float64
 2   orientation           500 non-null    int64  
 3   wall_U_value          500 non-null    float64
 4   roof_U_value          500 non-null    float64
 5   ground_U_value        500 non-null    float64
 6   construction_type     500 non-null    int64  
 7   ceiling_insulation    500 non-null    int64  
 8   u_factor              500 non-null    float64
 9   shgc                  500 non-null    float64
 10  window_to_wall_ratio  500 non-null    float64
 11  door_area             500 non-null    float64
 12  number_people         500 non-null    float64
 13  activity_level        500 non-null    float64
 14  P_equip               500 non-null    float64
 15

In [3]:
# Import failed simulations
failed_index = {}
for scenario in scenarios:
    path = '../simulations/' + scenario + '/failed_simulations.csv'
    failed_simulations = pd.read_csv(path)
    failed_simulations = failed_simulations.T
    failed_index[scenario] = failed_simulations.index[1:].values.astype(int)
    

In [4]:
# Exclude failed simulations
for key in inputs_dict.keys():
    inputs_dict[key] = inputs_dict[key].drop(failed_index[key])
    print('\n\n Scenario: ' + key)
    print(inputs_dict[key].info())



 Scenario: present-day
<class 'pandas.core.frame.DataFrame'>
Int64Index: 440 entries, 0 to 499
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   floor_area            440 non-null    float64
 1   volume                440 non-null    float64
 2   orientation           440 non-null    int64  
 3   wall_U_value          440 non-null    float64
 4   roof_U_value          440 non-null    float64
 5   ground_U_value        440 non-null    float64
 6   construction_type     440 non-null    int64  
 7   ceiling_insulation    440 non-null    int64  
 8   u_factor              440 non-null    float64
 9   shgc                  440 non-null    float64
 10  window_to_wall_ratio  440 non-null    float64
 11  door_area             440 non-null    float64
 12  number_people         440 non-null    float64
 13  activity_level        440 non-null    float64
 14  P_equip               440 non-null    float64
 15

In [5]:
inputs_dict['present-day'].head()

,floor_area,volume,orientation,wall_U_value,roof_U_value,ground_U_value,construction_type,ceiling_insulation,u_factor,shgc,window_to_wall_ratio,door_area,number_people,activity_level,P_equip,elec_gas,P_hvac,lighting_bulbs,infiltration_rate,type_house
key,,,,,,,,,,,,,,,,,,,,
0,52.960858,199.379961,45,3.35,0.174390,2.090000,1,1,1.372321,0.529771,0.114109,2.658326,9.0,134.877445,389.540048,Electric,1615.663658,3.0,0.445719,RDP
1,45.388881,122.802630,315,3.35,0.174033,1.218124,1,1,1.692861,0.593691,0.039233,2.077256,1.0,111.802371,47.488522,Electric,1075.336747,5.0,0.469739,RDP
2,46.656844,158.501730,180,3.35,0.174033,1.728701,1,1,1.237705,0.595338,0.130983,2.424799,8.0,95.473182,53.476870,Gas,1634.842032,4.0,0.374706,RDP
3,67.871616,281.885110,90,3.35,0.174390,1.218124,1,1,1.165207,0.571022,0.111862,3.381410,7.0,71.790752,263.902479,Electric,1775.467343,1.0,0.398482,RDP
4,49.317865,153.282543,270,3.35,31.500000,1.728701,1,0,1.372327,0.692081,0.100863,2.414570,3.0,74.558574,105.098427,Gas,662.219034,4.0,0.456430,RDP


In [6]:
# Save inputs
for key in inputs_dict.keys():
    path = '../results/simulations/' + key + '_inputs.csv'
    inputs_dict[key].to_csv(path)